In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib.request
import json
import csv
import sys
import time

In [2]:
current_data = pd.DataFrame(columns=['wikidata_id'])
current_data

,wikidata_id


In [88]:
'data/math_query.csv'

class WikidataCleaner():
    """cleans a csv file of wikidata nodes and finds 
       english wikipedia articles, if available.
    """
    def __init__(self):
        self.total_queries = 0
        self.total_written = 0
        self.total_skipped = 0
    
    def parse(self, input_file, output_file):   
        currently_written = 0
        try:
            f = open('data/math_c.csv', 'r')
            reader = csv.reader(f, delimiter=',')
            next(reader, None)
            existing_ids = [i[0] for i in reader]
        except:
            existing_ids = []
        with open(str(input_file), 'r') as infile, open(str(output_file), 'w') as outfile:
            base_url = 'http://www.wikidata.org/entity/'
            base_len = len(base_url)
            reader = csv.reader(infile, delimiter=',')
            writer = csv.writer(outfile, delimiter=',')
            writer.writerow(['wikidata_id', 'title', 'wikipedia_url', 'wikidata_url'])
            next(reader, None) 
            i = 0
#             while i < 10:
#                 row = next(reader)
            for i, row in enumerate(reader):
#                 print(existing_ids)
                data_url = row[0]
                data_id = data_url[base_len:]
                if data_id in existing_ids:
                    continue
                else:
                    self.total_queries += 1
                    existing_ids.append(data_id)
                    title = row[1]
    #                 time.sleep(1)
                    html = urllib.request.urlopen(data_url).read()
                    soup = BeautifulSoup(html, 'lxml')
                    wiki_data_dict = json.loads(str(soup.text))
                    try:
                        wikipedia_url = wiki_data_dict['entities'][str(data_id)]['sitelinks']['enwiki']['url']
                        writer.writerow([data_id, title, wikipedia_url, data_url])
                        self.total_written += 1
                        currently_written += 1
                        sys.stdout.write('\r'+ str(currently_written) + ' ' + str(self.total_written))
                    except:
                        self.total_skipped += 1
                    i += 1

In [89]:
parser = WikidataCleaner()

In [97]:
parser.parse('data/math_query.csv', 'data/math_c.csv')

17 65

KeyboardInterrupt: 

In [96]:
parser.total_skipped, parser.total_queries, parser.total_written

(11, 62, 48)

In [8]:
parser.total_skipped, parser.total_written

(2, 8)

In [7]:
data = pd.read_csv('data/math_query.csv')

In [144]:
data.head()

,item,itemLabel,linkTo
0,http://www.wikidata.org/entity/Q115646,Newton polygon,http://www.wikidata.org/entity/Q395
1,http://www.wikidata.org/entity/Q8366,algorithm,http://www.wikidata.org/entity/Q13636890
2,http://www.wikidata.org/entity/Q1908419,mathematical geography,http://www.wikidata.org/entity/Q102066
3,http://www.wikidata.org/entity/Q1908419,mathematical geography,http://www.wikidata.org/entity/Q131089
4,http://www.wikidata.org/entity/Q74304,least squares method,http://www.wikidata.org/entity/Q12483


In [151]:
data.loc[0,:].values

array(['http://www.wikidata.org/entity/Q115646', 'Newton polygon',
       'http://www.wikidata.org/entity/Q395'], dtype=object)

In [277]:
total_queries = 0
total_written = 0
total_skipped = 0

with open('data/math_query.csv', 'r') as infile, open('data/math_cleaned.csv', 'w') as outfile:
            base_url = 'http://www.wikidata.org/entity/'
            base_len = len(base_url)
            reader = csv.reader(infile, delimiter=',')
            writer = csv.writer(outfile, delimiter=',')
            writer.writerow(['wikidata_id', 'title', 'wikipedia_url', 'wikidata_url'])
            next(reader, None) 
            i = 0
            while i < 10:
                row = next(reader)
                total_queries += 1
                title = row[1]
                data_url = row[0]
            
                sys.stdout.write(data_url + '\n')
                
                data_id = data_url[base_len:]
                sys.stdout.write(data_id + '\n')
                time.sleep(1)
                html = urllib.request.urlopen(data_url).read()
                soup = BeautifulSoup(html, 'lxml')
                wiki_data_dict = json.loads(str(soup.text))
                try:
                    wikipedia_url = wiki_data_dict['entities'][str(data_id)]['sitelinks']['enwiki']['url']
                    writer.writerow([data_id, title, wikipedia_url, data_url])
                    total_written += 1
                except:
                    total_skipped += 1
                    print(wiki_data_dict['entities'][str(data_id)]['sitelinks'])
                i += 1

http://www.wikidata.org/entity/Q115646
Q115646
http://www.wikidata.org/entity/Q8366
Q8366
http://www.wikidata.org/entity/Q1908419
Q1908419
{'cawiki': {'site': 'cawiki', 'title': 'Geografia matemàtica', 'badges': [], 'url': 'https://ca.wikipedia.org/wiki/Geografia_matem%C3%A0tica'}, 'cbk_zamwiki': {'site': 'cbk_zamwiki', 'title': 'Geografía Matemática', 'badges': [], 'url': 'https://cbk-zam.wikipedia.org/wiki/Geograf%C3%ADa_Matem%C3%A1tica'}, 'dewiki': {'site': 'dewiki', 'title': 'Mathematische Geographie', 'badges': [], 'url': 'https://de.wikipedia.org/wiki/Mathematische_Geographie'}, 'eswiki': {'site': 'eswiki', 'title': 'Geografía matemática', 'badges': [], 'url': 'https://es.wikipedia.org/wiki/Geograf%C3%ADa_matem%C3%A1tica'}, 'frwiki': {'site': 'frwiki', 'title': 'Géographie mathématique', 'badges': [], 'url': 'https://fr.wikipedia.org/wiki/G%C3%A9ographie_math%C3%A9matique'}, 'glwiki': {'site': 'glwiki', 'title': 'Xeografía matemática', 'badges': [], 'url': 'https://gl.wikipedia.o

TypeError: Index(...) must be called with a collection of some kind, 'wikidata_id' was passed

In [113]:
wiki_data_dict['entities'][str(node)]['sitelinks']['enwiki']['url']


'https://en.wikipedia.org/wiki/Information_theory'

In [ ]:
def find_en_urls(df):
    base_url = 'http://www.wikidata.org/entity/'
    base_len = len(base_url)
    i = 0
    while i < 1000:
        title = df.loc['i']
        node_url = df[0]
        node_id = node_url[base_len:]
        html = urllib.request.urlopen(node_url).read()
        soup = BeautifulSoup(html, 'lxml')
        wiki_data_dict = json.loads(str(soup.text))
        try:
            return wiki_data_dict['entities'][str(node_id)]['sitelinks']['enwiki']['url'])
        except:
            skipped += 1

In [129]:
'data/math_query.csv'

class WikidataCleaner():
    """cleans a csv file of wikidata nodes and finds 
       english wikipedia articles, if available.
    """
    def __init__(self):
        self.total_queries = 0
        self.total_written = 0
        self.total_skipped = 0
    
    def parse(self, input_file, output_file):   
        currently_written = 0
        try:
            f = open('output_file', 'r')
            reader = csv.reader(f, delimiter=',')
            next(reader, None)
            existing_ids = [i[0] for i in reader]
        except:
            existing_ids = []
        with open(str(input_file), 'r') as infile, open(str(output_file), 'w') as outfile:
            wikidata_base = 'http://www.wikidata.org/entity/'
            wikipedia_base = 'https://en.wikipedia.org/wiki/'
            base_len = len(wikidata_base)
            reader = csv.reader(infile, delimiter=',')
            writer = csv.writer(outfile, delimiter=',')
            writer.writerow(['wikidata_id', 'title', 'wikipedia_url', 'wikidata_url'])
            next(reader, None) 
            i = 0
            while i < 10:
                row = next(reader)
#             for i, row in enumerate(reader):
                data_url = row[0]
                data_id = data_url[base_len:]
                if data_id in existing_ids:
                    continue
                else:
                    self.total_queries += 1
                    existing_ids.append(data_id)
                    title = row[1]
                    title = title.replace(' ', '_')
                    wikipedia_url = wikipedia_base + title
#                     print(wikipedia_url)
                    try:
                        html = urllib.request.urlopen(wikipedia_url).read()
                        print(wikipedia_url)
                        soup = BeautifulSoup(html, 'lxml')
#                         wiki_data_dict = json.loads(str(soup.text))
                        writer.writerow([data_id, title, wikipedia_url, data_url])
                        self.total_written += 1
                        currently_written += 1
                        sys.stdout.write('\r'+ str(currently_written) + ' ' + str(self.total_written))
                    except:
                        self.total_skipped +=1
                    i += 1

In [130]:
parse = WikidataCleaner()

In [131]:
parse.parse('data/math_query.csv', 'data/wiki_test.csv')

https://en.wikipedia.org/wiki/algorithm
1 1https://en.wikipedia.org/wiki/mathematical_geography
2 2https://en.wikipedia.org/wiki/least_squares_method
3 3https://en.wikipedia.org/wiki/transformation_geometry
4 4https://en.wikipedia.org/wiki/elementary_algebra
5 5https://en.wikipedia.org/wiki/mathematical_singularity
6 6https://en.wikipedia.org/wiki/information_theory
7 7https://en.wikipedia.org/wiki/product
8 8https://en.wikipedia.org/wiki/moment
9 9https://en.wikipedia.org/wiki/fractal
10 10

In [132]:
parse.total_skipped

0